# How to use scene and job

This tutorial demonstrates how to create a scene, and fill it from a speos file.
Then this demonstrates how to create a job from the scene, and run it.

In [1]:
from pathlib import Path
import time

from ansys.speos.core.kernel.job import ProtoJob
from ansys.speos.core.speos import Speos

# If using docker container
assets_data_path = Path("/app") / "assets"
# If using local server
# assets_data_path = Path().resolve().parent.parent / "tests" / "assets"
# If using a different path
# assets_data_path = Path("path/to/downloaded/example/assets")

Create connection with speos rpc server

In [2]:
speos = Speos(host="localhost", port=50098)

## Scene

Create an empty scene

In [3]:
my_scene = speos.client.scenes().create()

Load a file to fill the scene

In [4]:
speos_file = str(
    assets_data_path / "LG_50M_Colorimetric_short.sv5" / "LG_50M_Colorimetric_short.sv5"
)
my_scene.load_file(file_uri=speos_file)

Print scene data model

Here it is possible to see that the scene contains two surface sources, one irradiance sensor.

In [5]:
print(my_scene)

ansys.api.speos.scene.v2.Scene
{
    "name": "LG_50M_Colorimetric_short",
    "description": "From /app/assets/LG_50M_Colorimetric_short.sv5/LG_50M_Colorimetric_short.sv5",
    "part_guid": "ccf7cb1b-73f6-43d3-8445-0c06123d638f",
    "sources": [
        {
            "name": "Dom Source 2 (0) in SOURCE2",
            "source_guid": "f71597e8-9261-41e2-93a1-203bc02a34ec",
            "surface_properties": {
                "exitance_constant_properties": {
                    "geo_paths": [
                        {
                            "geo_path": "Solid Body in SOURCE2:2920204960/Face in SOURCE2:222",
                            "reverse_normal": false
                        }
                    ]
                }
            },
            "description": "",
            "metadata": {}
        },
        {
            "name": "Surface Source (0) in SOURCE1",
            "source_guid": "eec83682-4129-46e5-934c-548a6dc7d950",
            "surface_properties": {
              

## Job

Create a job for the first simulation. When loaded from a speos file, there is always only one
simulation in the scene.

In [6]:
# First create the protobuf message
job_message = ProtoJob(name="my_job")
job_message.scene_guid = my_scene.key  # The job needs a scene guid
job_message.simulation_path = (
    my_scene.get().simulations[0].name
)  # And needs to know which simulation in the scene is involved.
job_message.job_type = ProtoJob.Type.CPU  # Choose type of job, can also be GPU.
job_message.direct_mc_simulation_properties.automatic_save_frequency = 1800
job_message.direct_mc_simulation_properties.stop_condition_rays_number = (
    200000  # Stop condition, here 200000 rays will be sent.
)

# Create the JobLink
job_link = speos.client.jobs().create(message=job_message)

Start the job

In [7]:
job_link.start()

Verify state of the job

In [8]:
job_link.get_state()

state: RUNNING

Wait that the job is finished

In [9]:
job_state_res = job_link.get_state()
while (
    job_state_res.state != ProtoJob.State.FINISHED
    and job_state_res.state != ProtoJob.State.STOPPED
    and job_state_res.state != ProtoJob.State.IN_ERROR
):
    time.sleep(2)

    job_state_res = job_link.get_state()

Retrieve results of the job

Two results are generated : the result of irradiance sensor:
"ASSEMBLY1.DS (0).Dom Irradiance Sensor (0).xmp" and the simulation report in html

In [10]:
results = job_link.get_results().results
print(results)

[upload_response {
  info {
    uri: "90697b90-1e67-467e-a23e-36de528e3759"
    file_name: "ASSEMBLY1.DS (0).Dom Irradiance Sensor (0).xmp"
    file_size: 1437361
  }
  upload_duration {
    nanos: 2263824
  }
}
, upload_response {
  info {
    uri: "55fe9812-640e-4756-9d35-b6d6f8ace115"
    file_name: "ASSEMBLY1.DS (0).html"
    file_size: 508290
  }
  upload_duration {
    nanos: 1420347
  }
}
]


Once no more needed: delete the job

In [11]:
job_link.delete()

When loading a speos file into a scene, this creates many objects
(source templates, sensor templates, vop template, sop templates).
Then at the end of the example, we just clean all databases

In [12]:
for item in (
    speos.client.scenes().list()
    + speos.client.simulation_templates().list()
    + speos.client.sensor_templates().list()
    + speos.client.source_templates().list()
    + speos.client.intensity_templates().list()
    + speos.client.spectrums().list()
    + speos.client.vop_templates().list()
    + speos.client.sop_templates().list()
    + speos.client.parts().list()
    + speos.client.bodies().list()
    + speos.client.faces().list()
):
    item.delete()